# Importing libraries and supressing the visualization error

In [15]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

# Set default font to DejaVu Sans
rcParams['font.sans-serif'] = ['DejaVu Sans']

In [17]:
import pandas as pd 
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from scipy.stats import friedmanchisquare
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import LassoCV, Ridge
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.preprocessing import StandardScaler
from scipy.stats import friedmanchisquare
from sklearn.model_selection import KFold

 # Reading all the data

In [3]:
df_all=pd.read_csv('data_compiled.csv')

In [4]:
df_all.drop(['Unnamed: 0'],axis=1, inplace=True)

In [5]:
df_all=df_all.dropna()

In [6]:
#categorical_features=['Participant','Gender'] categorical features are already one hot encoded

In [7]:
other_features=['HR_Mean', 'HR_Max', 'HR_Std', 'HR_Skew', 'HR_Q1G',
       'HR_Q3G', 'EDA_Mean', 'EDA_Max', 'EDA_Std', 'EDA_Skew', 'EDA_Q1G',
       'EDA_Q3G', 'TEMP_Mean', 'TEMP_Max', 'TEMP_Std', 'TEMP_Skew', 'TEMP_Q1G',
       'TEMP_Q3G', 'ACC_Mean', 'ACC_Max', 'ACC_Std', 'ACC_Skew', 'ACC_Q1G',
       'ACC_Q3G', 'BVP_Mean', 'BVP_Max', 'BVP_Std', 'BVP_Skew', 'BVP_Q1G',
       'BVP_Q3G', 'PeakEDA', 'maxHRV', 'minHRV', 'medianHRV', 'SDNN', 'NN50',
       'pNN50', 'RMSSD', 'calories2hr', 'protien2hr', 'sugar2hr', 'carbs2hr',
       'protien8hr', 'sugar8hr', 'carbs8hr', 'calories24hr', 'protien24hr',
       'sugar24hr', 'carbs24hr', 'Eat', 'Eatcnt2hr', 'Eatcnt8hr', 'Eatcnt24hr',
       'Eatmean2hr', 'Eatmean8hr', 'Eatmean24hr', 'Minfrommid', 'Hourfrommid',
       'EDA_Min', 'calories8hr', 'TEMP_Min', 'BVP_Min', 'ACC_Min', 'HR_Min',
       'HR_Mean_Historical', 'HR_Std_Historical', 'ACC_Mean_Historical',
       'ACC_Std_Historical', 'Activity_Bouts', 'Activity24',
       'Activity1h']

In [9]:
categorical_features=['Participant','Gender']

In [8]:
all_sleep_features=['total_sleep_time','percent_time_asleep','waso','sleep_onset_latency',
                    'number_wake_bouts', 'stage_0_count','stage_2_count','stage_3_count',
                    'stage_5_count','stage_0_count_log','total_sleep_time_log',
                    'percent_time_asleep_log','stage_0_count_boxcox','stage_0_count_yeojohnson',
                    'stage_0_count_rank','stage_0_count_standardized','total_sleep_timelog',
                    'percent_time_asleeplog','wasolog','sleep_onset_latencylog','number_wake_boutslog',
                    'stage_0_countlog','stage_1_countlog','stage_2_countlog','stage_3_countlog','stage_5_countlog',
                    'total_sleep_time_boxcox','total_sleep_time_yeojohnson','total_sleep_time_rank','percent_time_asleep_boxcox',
                    'percent_time_asleep_yeojohnson','percent_time_asleep_rank','waso_boxcox','waso_yeojohnson','waso_rank',
                    'sleep_onset_latency_yeojohnson','sleep_onset_latency_rank','number_wake_bouts_boxcox','number_wake_bouts_yeojohnson',
                    'number_wake_bouts_rank','stage_1_count_boxcox','stage_1_count_yeojohnson','stage_1_count_rank','stage_2_count_boxcox',
                    'stage_2_count_yeojohnson','stage_2_count_rank','stage_3_count_boxcox','stage_3_count_yeojohnson','stage_3_count_rank',
                    'stage_5_count_boxcox','stage_5_count_yeojohnson','stage_5_count_rank','stage_0_count_percent','stage_1_count_percent',
                    'stage_2_count_percent','stage_3_count_percent','stage_5_count_percent','stage_1_count_percent_diff_from_recommended',
                    'stage_2_count_percent_diff_from_recommended','stage_3_count_percent_diff_from_recommended',
                    'stage_5_count_percent_diff_from_recommended']

Let us use only the selected transformed features 
the table defining them is given here
| Feature                          | Original | Log Transformation | Box-Cox Transformation | Yeo-Johnson Transformation | Rank-Based Inverse Normal Transformation |
|----------------------------------|----------|---------------------|------------------------|----------------------------|------------------------------------------|
| total_sleep_time                 | ✖        | ✔                   | ✖                      | ✖                          | ✔                                        |
| percent_time_asleep              | ✖        | ✔                   | ✖                      | ✖                          | ✔                                        |
| waso                             | ✔        | ✖                   | ✖                      | ✖                          | ✖                                        |
| sleep_onset_latency              | ✖        | ✖                   | ✖                      | ✖                          | ✔                                        |
| number_wake_bouts                | ✔        | ✖                   | ✖                      | ✖                          | ✖                                        |
| stage_0_count                    | ✖        | ✖                   | ✖                      | ✖                          | ✔                                        |
| stage_1_count                    | ✖        | ✖                   | ✖                      | ✖                          | ✔                                        |
| stage_2_count                    | ✖        | ✖                   | ✖                      | ✖                          | ✔                                        |
| stage_3_count                    | ✖        | ✖                   | ✖                      | ✔                          | ✖                                        |
| stage_5_count                    | ✖        | ✖                   | ✖                      | ✔                          | ✖                                        |

In [9]:
transformed_sleep_features=['waso','number_wake_bouts','stage_0_count_log','total_sleep_time_log',
                    'percent_time_asleep_log', 'total_sleep_timelog', 'percent_time_asleeplog','total_sleep_time_rank',
                    'percent_time_asleep_rank','sleep_onset_latency_rank','stage_1_count_rank','stage_3_count_yeojohnson',
                    'stage_2_count_rank','stage_3_count_rank','stage_5_count_yeojohnson','stage_0_count_percent','stage_1_count_percent',
                    'stage_2_count_percent','stage_3_count_percent','stage_5_count_percent','stage_1_count_percent_diff_from_recommended',
                    'stage_2_count_percent_diff_from_recommended','stage_3_count_percent_diff_from_recommended',
                    'stage_5_count_percent_diff_from_recommended']

In [10]:
unmodified_sleep_features= ['total_sleep_time','percent_time_asleep','waso','sleep_onset_latency',
                    'number_wake_bouts', 'stage_0_count','stage_2_count','stage_3_count',
                    'stage_5_count']


In [11]:
feature_sets = {
    'other_features': other_features,
    'other_features + unmodified_sleep_features': other_features + unmodified_sleep_features,
    'other_features + transformed_sleep_features': other_features + transformed_sleep_features,
    'other_features + all_sleep_features': other_features + all_sleep_features
}


# Getting the results for different feature sets and models

In [12]:
import pandas as pd
import numpy as np
from scipy.stats import friedmanchisquare
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import LassoCV, Ridge
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.preprocessing import StandardScaler
from scipy.stats import friedmanchisquare
from sklearn.model_selection import KFold
from tqdm import tqdm

In [13]:
# Define models
models = {
    'DT': DecisionTreeRegressor(),
    'SVM': SVR(),
    'RF': RandomForestRegressor(),
    'KNN': KNeighborsRegressor(),
    'LassoCV': LassoCV(),
    'Ridge': Ridge(),
    'AdaBoost': AdaBoostRegressor(),
    'XGBOOST': XGBRegressor()
}

In [ ]:
results = []

kf = KFold(n_splits=10, shuffle=True, random_state=42)
maes=[]
# Outer loop for each model
for name, model in tqdm(models.items(), desc='Models', position=1,leave=False):
    for set_name, feature_set in feature_sets.items():
        X_subset = df_all[feature_set]
        y=df_all['IG']
        
        for fold, (train_index, test_index) in enumerate(tqdm(kf.split(X_subset), desc=f'{name} - {set_name} Folds', leave=False, position=0, total=kf.get_n_splits()), 1):
            X_train, X_test = X_subset.iloc[train_index], X_subset.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            mae = mean_absolute_error(y_test, y_pred)
            mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
            r2 = r2_score(y_test, y_pred)
            n, p = X_test.shape
            adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
            msle = mean_squared_error(y_test, y_pred, squared=False)
            ev = explained_variance_score(y_test, y_pred)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            maes.append(mae)

            results.append({
                'Model': name,
                'Feature Set': set_name,
                'Fold': fold,
                'MAE': mae,
                'MAPE': mape,
                'R2': r2,
                'Adjusted R2': adj_r2,
                'MSLE': msle,
                'Explained Variance': ev,
                'RMSE': rmse
            })
        mean_mae = np.mean(maes)
        std_mae = np.std(maes)
        maes=[]
        print(f'Average MAE for {name} on {set_name}: {mean_mae:.4f} ± {std_mae:.4f}')

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Calculate mean and standard deviation for MAE
mae_metrics = results_df.groupby(['Model', 'Feature Set'])['MAE'].agg(['mean', 'std'])

# Print the average MAE ± STD for each model and feature set
print("\nAverage MAE ± STD for each model and feature set:")
for idx, row in mae_metrics.iterrows():
    print(f"{idx[0]} - {idx[1]}: {row['mean']:.4f} ± {row['std']:.4f}")

# Save the results to a CSV file
results_df.to_csv('model_performance_metrics_regression.csv', index=False)

print("Model performance metrics have been saved to 'model_performance_metrics_regression.csv'.")

Average MAE for DT on other_features: 8.1355 ± 0.3267


Average MAE for DT on other_features + unmodified_sleep_features: 6.8294 ± 0.2752


Average MAE for DT on other_features + transformed_sleep_features: 6.6071 ± 0.2143


                                                                                               
Models:  12%|█▎        | 1/8 [01:13<08:33, 73.41s/it]

Average MAE for DT on other_features + all_sleep_features: 6.6047 ± 0.3250


Average MAE for SVM on other_features: 16.1494 ± 0.3992


SVM - other_features + unmodified_sleep_features Folds:  80%|████████  | 8/10 [03:41<00:54, 27.41s/it]

Let us see what would be the accuracy of RF model if only sleep features are used. (we use the 70/30) split.